In [3]:
import pandas as pd
from lifetimes import BetaGeoFitter, GammaGammaFitter

In [4]:
df_customers = pd.read_csv('data/raw/customers.csv')
df_orders =  pd.read_csv('data/raw/orders.csv')

In [13]:
def customer_lifetime_value_analysis(df_customers, df_orders):
    """
    Calcule et analyse le Customer Lifetime Value (CLV)
    - CLV historique
    - CLV prédictif simple
    - Segmentation des clients
    """
    df_customers = df_customers.copy()
    df_orders = df_orders.copy()

    # Historical CLV 
    clv_hist = df_orders.groupby('customer_id')['total_amount'].sum().reset_index()
    clv_hist.rename(columns={'total_amount': 'clv_historical'}, inplace=False)

    # Commands frequency
    freq = df_orders.groupby('customer_id')['order_id'].nunique().reset_index()
    freq.rename(columns={'order_id': 'purchase_frequency'}, inplace=True)

    # Average Order Value
    avg_order_value = (df_orders.groupby('customer_id')['total_amount'].sum() /
                       df_orders.groupby('customer_id')['order_id'].nunique()).reset_index()
    avg_order_value.rename(columns={0: 'avg_order_value'}, inplace=True)

    df_clv = df_customers.merge(clv_hist, on='customer_id', how='left')
    df_clv = df_clv.merge(freq, on='customer_id', how='left')
    df_clv = df_clv.merge(avg_order_value, on='customer_id', how='left')

    # Basic CLV predective
    estimated_lifespan_years = 3
    df_clv['clv_predicted'] = (
        df_clv['avg_order_value'] *
        df_clv['purchase_frequency'] *
        estimated_lifespan_years
    )

    # Client segmentation
    df_clv['segment'] = pd.qcut(
        df_clv['clv_predicted'],
        q=4,
        labels=['Low Value', 'Mid-Low', 'Mid-High', 'High Value']
    )

    return df_clv

In [14]:
customer_lifetime_value_analysis(df_customers, df_orders)

,customer_id,first_name,last_name,email,phone,birth_date,registration_date,country,city,postal_code,address,segment,newsletter_subscribed,total_amount,purchase_frequency,avg_order_value,clv_predicted
0,CUST_10000,Jeanne,Imbert,fernandesnath@example.com,06 32 24 48 76,1957-11-29,2023-12-03,France,Fournier,84486,"821, chemin Élise Morel, 18211 Da Silva-sur-Paul",High Value,False,17385.41,5.0,3477.082000,52156.23
1,CUST_10001,Pénélope,Lagarde,cboucher@example.org,0222984672,1975-10-14,2024-05-03,France,Gosselinnec,60377,"63, avenue Véronique Marie, 44903 Torres-sur-Mer",Mid-Low,True,5967.71,2.0,2983.855000,17903.13
2,CUST_10002,Olivier,Lacombe,constancelanglois@example.org,+33 (0)3 90 48 81 86,1986-12-24,2024-10-13,France,Saint Alain,50985,"86, rue de Delorme, 85279 Humbert",Low Value,False,1775.89,1.0,1775.890000,5327.67
3,CUST_10003,Eckhart,Scholl,bruno15@example.com,+49(0) 035346152,1975-02-17,2025-06-25,Germany,Neustadt am Rübenberge,14114,"Ditschlerinstr. 6/6, 88150 Saulgau",Low Value,False,5226.09,4.0,1306.522500,15678.27
4,CUST_10004,Jose Antonio,Caparrós,oscarpalacio@example.net,+34 848 74 33 82,1995-03-05,2025-01-10,Spain,Barcelona,46063,"Glorieta de Benita Álamo 57 Apt. 57 , Lleida, ...",Mid-High,True,9036.55,2.0,4518.275000,27109.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,CUST_14995,Susanne,Benoit,zcollet@example.net,+33 4 49 73 19 65,1991-09-23,2024-10-04,France,Sainte Adélaïde,67958,"79, rue de Germain, 30927 Delahaye-les-Bains",Mid-Low,False,8066.30,3.0,2688.766667,24198.90
4996,CUST_14996,Lindsay,Rodriguez,barrongary@example.com,281.622.9288x27541,1960-07-20,2025-02-12,United States,Martinezberg,33108,"09600 Randall Stream Apt. 198, Reillyton, MA 4...",High Value,False,17915.57,5.0,3583.114000,53746.71
4997,CUST_14997,Odette,Vallet,marineperrin@example.com,02 96 67 25 53,1957-06-01,2023-01-04,France,Monnier,44657,"989, boulevard Marine Hoareau, 15905 Delattreb...",Mid-Low,True,8159.87,2.0,4079.935000,24479.61
4998,CUST_14998,Faruk,Eckbauer,kuniberthartmann@example.org,0709814847,1964-07-24,2025-10-31,Germany,Dresden,97355,"Marc-Budig-Straße 8/3, 45966 Ebermannstadt",High Value,True,14177.90,7.0,2025.414286,42533.70


In [17]:
df_items = pd.read_csv('data/raw/order_items.csv')

In [25]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

def market_basket_analysis(df_orders):
    """
    Analyse du panier d'achat :
    - Trouve les produits fréquemment achetés ensemble
    - Génère des règles d'association
    """
    # Préparation du panier
    basket = df_orders.groupby(['order_id', 'product_id'])['product_id'] \
        .count().unstack().fillna(0)
    
    # One-hot encoding booléen
    basket = basket.applymap(lambda x: x > 0)

    # Itemsets fréquents
    frequent_itemsets = apriori(basket, min_support=0.01, use_colnames=True)

    # Règles d'association
    rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)
    rules = rules.sort_values('lift', ascending=False).reset_index(drop=True)

    return frequent_itemsets, rules


In [26]:
market_basket_analysis(df_items)

(     support     itemsets
 0    0.01370  (PROD_5000)
 1    0.01370  (PROD_5001)
 2    0.01555  (PROD_5002)
 3    0.01445  (PROD_5003)
 4    0.01430  (PROD_5004)
 ..       ...          ...
 195  0.01320  (PROD_5195)
 196  0.01570  (PROD_5196)
 197  0.01425  (PROD_5197)
 198  0.01445  (PROD_5198)
 199  0.01565  (PROD_5199)
 
 [200 rows x 2 columns],
 Empty DataFrame
 Columns: [antecedents, consequents, antecedent support, consequent support, support, confidence, lift, representativity, leverage, conviction, zhangs_metric, jaccard, certainty, kulczynski]
 Index: [])